In [63]:
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import tensorflow.keras
import keras.metrics
from sklearn.cluster import DBSCAN
from sklearn.feature_selection import RFECV, RFE
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Input
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils import plot_model
from keras.optimizers import SGD
from keras.models import Model
from imblearn.under_sampling import RandomUnderSampler
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
import featuretools as ft
from sklearn.preprocessing import StandardScaler
import random 


In [64]:
path = "PTSD.xlsx"
df = pd.read_excel(path)
df = df[~df["PCL_Strict3"].isna()]

In [65]:
features = ["age", "highschool_diploma",  "dyslexia", "ADHD", "T1Acc1t", "T1Acc1n", "T1bias", "phq1", "lot1", "trait1",
               "state1", "PCL1",  "PCL_Broad1", "PCL_Strict1", "phq2", "lot2", "trait2", "state2", "PCL2", "PCL_Broad2", 
                 "PCL_Strict2", "cd_risc1", "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                 "religion1", "emotional_support1","instrumental_support1", "self_distraction1", "denial1", 
               "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1", "active_coping2", "planning2",
                "positive_reframing2", "acceptance2", "humor2", "religion2", "emotional_support2", "instrumental_support2", 
                 "self_distraction2", "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                 "trauma_history8_1", "HML_5HTT", "HL_MAOA", "HML_NPY", "COMT_Ranked", "COMT_Hap1_recode", 
               "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "bad_features"]

In [67]:
bad_features = ["T1ETBE", "T1bias", "state1", "state2", "trait1", "trait2", "phq1", "phq2", "PCL1", "PCL2",
                "denial1", "substance_use1", "self_blame1", "denial2", "substance_use2","self_blame2",
                         "trauma_history8_1"]
numerical_features = [ "T1ETBE", "T1Acc1t", "T1Acc1n", "T1bias", "T2Acc1t","T2Acc1n", "T2bias", "state1", "state2",
                        "trait1", "trait2", "lot1", "lot2", "phq1", "phq2", "cd_risc1", "PCL1", "PCL2", "bad_features"]
categorical_features = ["age", "highschool_diploma",  "dyslexia", "ADHD", "T1Acc1t", "T1Acc1n", "T1bias", "phq1", "lot1", "trait1",
               "state1", "PCL1",  "PCL_Broad1", "PCL_Strict1", "phq2", "lot2", "trait2", "state2", "PCL2", "PCL_Broad2", 
                 "PCL_Strict2", "cd_risc1", "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                 "religion1", "emotional_support1","instrumental_support1", "self_distraction1", "denial1", 
               "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1", "active_coping2", "planning2",
                "positive_reframing2", "acceptance2", "humor2", "religion2", "emotional_support2", "instrumental_support2", 
                 "self_distraction2", "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                 "trauma_history8_1", "HML_5HTT", "HL_MAOA", "HML_NPY", "COMT_Ranked", "COMT_Hap1_recode", 
               "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "Ashken_scale", "Sephar_scale", "Unknown"]

df['bad_features'] = (df > df.mean())[bad_features].sum(axis=1)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
df[numerical_features] = imp.fit_transform(df[numerical_features])

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df[categorical_features] = imp.fit_transform(df[categorical_features])

pca = PCA(n_components = 1)
df["pcls"] = pca.fit_transform(df[["PCL1",  "PCL_Broad1", "PCL_Strict1"]])
df.drop(["PCL1",  "PCL_Broad1", "PCL_Strict1"], axis=1, inplace=True)
features.append("pcls")
features.remove("PCL1")
features.remove("PCL_Broad1")
features.remove("PCL_Strict1")


X = df[features]


ss = StandardScaler()
X = ss.fit_transform(X)


Y = df["PCL_Strict3"]

In [68]:
rand_item = eval("['PCL2', 'positive_reframing1', 'trauma_history8_1', 'lot2', 'T1bias', 'emotional_support1', 'self_blame1', 'PCL2']")

pca = PCA(n_components = 2)
data_transformed = df[rand_item]
data_transformed = pca.fit_transform(data_transformed)

db = DBSCAN(eps=0.5, min_samples=10, leaf_size=30).fit(X)
X = np.hstack((db.labels_.reshape(-1,1), X))


rand_item = eval("['instrumental_support1', 'COMT_Hap1_recode', 'dyslexia', 'HL_MAOA', 'lot1', 'active_coping2', 'ADHD']")

pca = PCA(n_components = 2)
data_transformed = df[rand_item]
data_transformed = pca.fit_transform(data_transformed)

db = DBSCAN(eps=0.7, min_samples=10, leaf_size=30).fit(X)
X = np.hstack((db.labels_.reshape(-1,1), X))


rand_item = eval("['COMT_Hap1_LvsMH', 'venting2', 'positive_reframing2', 'COMT_Ranked', 'behavioral_disengagement2', 'trait1', 'HL_MAOA', 'HL_MAOA']")

pca = PCA(n_components = 2)
data_transformed = df[rand_item]
data_transformed = pca.fit_transform(data_transformed)

db = DBSCAN(eps=0.5, min_samples=10, leaf_size=30).fit(X)
X = np.hstack((db.labels_.reshape(-1,1), X))


rand_item = eval("['PCL_Strict2', 'emotional_support2', 'PCL_Broad2', 'emotional_support1', 'COMT_Hap1_LvsMH', 'HML_FKBP5', 'lot1', 'COMT_Hap1_LvsMH', 'COMT_Hap2_recode', 'state1']")

pca = PCA(n_components = 2)
data_transformed = df[rand_item]
data_transformed = pca.fit_transform(data_transformed)

db = DBSCAN(eps=0.6, min_samples=10, leaf_size=30).fit(X)
X = np.hstack((db.labels_.reshape(-1,1), X))


rand_item = eval("['phq2', 'substance_use1', 'instrumental_support2', 'substance_use1', 'trait1']")

pca = PCA(n_components = 2)
data_transformed = df[rand_item]
data_transformed = pca.fit_transform(data_transformed)

db = DBSCAN(eps=0.5, min_samples=5, leaf_size=30).fit(X)
X = np.hstack((db.labels_.reshape(-1,1), X))


In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state=271828, stratify=Y)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.1, random_state=271828, stratify=y_train)


In [70]:
def create_data(X_train, y_train):
    X_train_3 = X_train[y_train==1]
    y_train_3 = y_train[y_train==1]
    X_train_4 = X_train[y_train==0][:15:]
    y_train_4 = y_train[y_train==0][:15:]
    X_train_5 = np.vstack((X_train_4, X_train_3))
    y_train_5 =  np.hstack((y_train_4, y_train_3))
    sm = SMOTE(random_state=27)
    X_train_6, y_train_6 = sm.fit_sample(X_train_5, y_train_5.ravel())
    X_train_6 = X_train_6[y_train_6==0]
    y_train_6 = y_train_6[y_train_6==0]
    return X_train_6, y_train_6

In [71]:
def regressions(X_train, y_train, X_test):
    gbr = GradientBoostingRegressor()
    gbr.fit(X_train, y_train)
    
    X_train = np.hstack((X_train, gbr.predict(X_train).reshape(-1, 1)))
    
    X_test= np.hstack((X_test, gbr.predict(X_test).reshape(-1, 1)))
    return X_train, X_test

In [72]:
good_ones= eval('[[[59], [57], [11], [40], [60], [11], [43], [20], [51], [56], [46], [39], [15], [13], [54]], [[12], [13], [43], [15], [43], [32], [3], [61], [51], [58], [31], [13], [0], [12], [4]], [[56], [16], [29], [49], [1], [19], [4], [1], [12], [59], [15], [48], [18], [58], [35]]]')

In [77]:
features_groups = []
k = 1
for j in range(k):
    rand_item = [[random.randrange(X.shape[1])] for i in range(45)]
    #features_groups.append(rand_item)
repr(features_groups)

rand_item = [[i] for i in range(X.shape[1])]
features_groups.append(rand_item)

In [78]:
len(features_groups)

1

In [79]:
preds = []
trues= []

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)
cvscores = []
y_train_2 = np.array(y_train_2)
X_train_2 = np.array(X_train_2)

for class_weight in [{1:1,  0:1.1}]:
    for num_smote in [0.99]:
        for lim in [0.5]:
            for first_layer in [55]:
                for loops in [60]:
                    for each_layer in [20]:
                        n_pca = 0.99
                        first_layer = first_layer
                        each_layer = each_layer
                        num_smote = num_smote
                        loops = loops
                        train_scores_f = []
                        train_scores_p = []
                        train_scores_r = []
                        
                        scores_f = []
                        scores_p = []
                        scores_r = []
                        print("\n\nlim", lim, "\nfirst_layer", first_layer, 
                                  "\neach_layer", each_layer, "\nnum_smote", num_smote,
                              "\nloops", loops, "\nclass_weight", class_weight)
                        for train, test in kfold.split(X_train_2, y_train_2):
                            
                            y_train_pred = np.zeros_like(y_train_2[train], dtype= 'float32').reshape(-1, 1)

                            y_pred = np.zeros_like(y_train_2[test], dtype = 'float32').reshape(-1, 1)
                            for feature_group in features_groups:
                                
                                #feature_group = [features.index(i) for i in feature_group]
                                
                                
                                
                                X_train_chosen_features =  X_train_2[train, feature_group].T
                                y_train_chosen_features = y_train_2[train]
                                
                                
                                X_test_chosen_features = X_train_2[test, feature_group].T
                                
                                
                                
                                rfe =  RFE(RandomForestClassifier(n_estimators=100), 14).fit(X_train_chosen_features,
                                                                                             y_train_chosen_features)
                                X_train_after_rfe = rfe.transform(X_train_chosen_features)
                                X_test_after_rfe = rfe.transform(X_test_chosen_features)
                                
                                
                                tmp_x, tmp_y = create_data(X_train_after_rfe, y_train_chosen_features)
                                created_X = tmp_x
                                created_y = tmp_y
                                for i in range(loops):
                                    aug_X_train, aug_y_train = shuffle(X_train_after_rfe, y_train_chosen_features)
                                    tmp_x, tmp_y = create_data(aug_X_train, aug_y_train)
                                    created_X = np.vstack((created_X, aug_X_train))
                                    created_y = np.hstack((created_y, aug_y_train))
                                aug_X_train = np.vstack([created_X, aug_X_train])
                                aug_y_train =  np.hstack([created_y, aug_y_train])

                                sm = SMOTE()
                                X_train_res, y_train_res = sm.fit_sample(aug_X_train, aug_y_train.ravel())
                                
                                
                                n_cols = X_train_res.shape[1]
                                model = Sequential()

                                model.add(Dense(first_layer, activation='elu'))
                                model.add(Dropout(0.5))
                              
                                model.add(Dense(7, activation='elu'))
                                model.add(Dropout(0.5))
                                           
                                model.add(Dense(7, activation='elu'))
                                model.add(Dropout(0.5))
                                
                                model.add(Dense(7, activation='elu'))
                                model.add(Dropout(0.5))
                                model.add(Dense(1, activation='sigmoid'))

                                model.compile(optimizer='adam', 
                                                  loss='binary_crossentropy')
      
                                # Fit the model
                                callbacks = [EarlyStopping(monitor='val_loss', patience=3)]
                                model.fit(X_train_res, y_train_res, epochs = 250, verbose=0, callbacks=callbacks)
                                # evaluate the model
                                y_pred +=  model.predict(X_test_after_rfe)
                                
                                # training error
                                y_train_pred +=  model.predict(X_train_after_rfe) 
                            
                            
                            y_pred = (y_pred/k) 
                            preds.extend(y_pred)
                            trues.extend(y_train_2[test])
                            y_pred = y_pred > lim
                            
                            s_f = f1_score(y_train_2[test], y_pred)
                            s_p = precision_score(y_train_2[test], y_pred)
                            s_r = recall_score(y_train_2[test], y_pred)
                            print("\tscores f1", (s_f))
                            print("\tscores p", (s_p))
                            print("\tscores r", (s_r))
                            scores_f.append(s_f)
                            scores_p.append(s_p)
                            scores_r.append(s_r)
                            
                            y_train_pred = (y_train_pred/k) > lim
                            train_s_f = f1_score(y_train_2[train], y_train_pred)
                            train_s_p = precision_score(y_train_2[train], y_train_pred)
                            train_s_r = recall_score(y_train_2[train], y_train_pred)
                            print("\tscores f1 train", (train_s_f))
                            print("\tscores p train", (train_s_p))
                            print("\tscores r train", (train_s_r))
                            train_scores_f.append(train_s_f)
                            train_scores_p.append(train_s_p)
                            train_scores_r.append(train_s_r)

                        print("mean scores f1", np.mean(scores_f))
                        print("mean scores p", np.mean(scores_p))
                        print("mean scores r", np.mean(scores_r))
                        
                        print("mean scores f1 train", np.mean(train_scores_f))
                        print("mean scores p train", np.mean(train_scores_p))
                        print("mean scores r train", np.mean(train_scores_r))



lim 0.5 
first_layer 55 
each_layer 20 
num_smote 0.99 
loops 60 
class_weight {1: 1, 0: 1.1}
	scores f1 0.14285714285714288
	scores p 0.16666666666666666
	scores r 0.125
	scores f1 train 0.8813559322033899
	scores p train 0.8387096774193549
	scores r train 0.9285714285714286
	scores f1 0.0
	scores p 0.0
	scores r 0.0
	scores f1 train 0.9206349206349206
	scores p train 0.8529411764705882
	scores r train 1.0


In [ ]:
preds = np.array(preds)
trues = np.array(trues)

In [ ]:
trues[np.where(preds > 0.)[0]].mean()

In [ ]:
precision_score(y_train_2[test], model.predict(X_test_cv) > 0.95)

In [ ]:
X_test_cv.shape